<a href="https://colab.research.google.com/github/sjhjrcode/NLP/blob/master/NLP_for_Eli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sjhjrcode/NLP



Cloning into 'NLP'...
remote: Enumerating objects: 881, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 881 (delta 30), reused 43 (delta 5), pack-reused 806
Receiving objects: 100% (881/881), 415.85 MiB | 13.36 MiB/s, done.
Resolving deltas: 100% (406/406), done.
Checking out files: 100% (403/403), done.


In [2]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash


200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2022-11-16 17:41:35 (81.8 MB/s) - written to stdout [1191/1191]

Installing PySpark 3.2.1 and Spark NLP 4.2.3
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.3
     |████████████████████████████████| 281.4 MB 46 kB/s 
     |████████████████████████████████| 648 kB 66.2 MB/s 
     |████████████████████████████████| 198 kB 65.8 MB/s 


In [3]:
import sparknlp
 
spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)

Spark NLP version 4.2.3
Apache Spark version: 3.2.1


In [4]:
from sparknlp.base import LightPipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

loaded_ner_model = NerDLModel.load("NLP/outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


TypeError: ignored

In [ ]:
text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")